In [ ]:
#Import necessary packages.

from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import he_uniform
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
#Load functional signatures of premises.

X = np.load('Fun.npy')

(num_contexts, num_fun) = X.shape
print(X.shape)

In [ ]:
#Form context distributions for each functor symbol.

output_data = np.zeros((num_fun, num_fun), dtype = 'float32')

for n in range(num_fun):
    for m in range(num_contexts):
        if X[m, n] != 0:
            output_data[n] += X[m]
    output_data[n] /= np.sum(output_data[n])
    
np.save('Output_data', output_data)

In [ ]:
#Define input and output data.

input_data = np.identity(num_fun)
#output_data = np.load('Output_data.npy')

In [ ]:
#Embedding model.

model = Sequential()
model.add(Dense(256, kernel_initializer = he_uniform(), activation = 'tanh', input_shape = (num_fun,)))
model.add(Dense(num_fun, kernel_initializer = he_uniform(), activation = 'softmax'))
model.compile(optimizer = RMSprop(decay = 1e-8), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

history = model.fit(np.identity(num_fun), output_data, epochs = 150, batch_size = 2048, shuffle = True, verbose = 0)

history_dict = history.history
loss_values = history_dict['loss']
acc_values = history_dict['acc']

epochs = range(1, len(acc_values) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.clf()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
#Test context distribution network.

test_loss, test_acc = model.evaluate(np.identity(num_fun), output_data, verbose = 0)
print('Test loss:', str(round(test_loss, 4)) + '.')
print('Test accuracy:', str(round(100*test_acc, 2)) + '%.')

In [ ]:
#Save embedded functional signatures.

weights = model.layers[0].get_weights()
X = np.tanh(np.dot(X, weights[0]) + weights[1])
print(X.shape)

np.save('Conj_tokens', X[:32524])
np.save('Ax_tokens', X[32524:])